Import PySpark modules

In [1]:
from pyspark import SparkConf
from pyspark import SparkContext

Import named-entity recognition library.

In [2]:
import geograpy

Create a SparkContext that uses the YARN resource manager and specifies a name for the application

In [3]:
conf = SparkConf()
conf.setMaster('yarn-client')
conf.setAppName('spark-nltk')
sc = SparkContext(conf=conf)

Specify the location of the data on the cluster nodes

In [24]:
data = sc.wholeTextFiles('file:///usr/share/nltk_data/corpora/state_union/')

Import the NLTK package

In [5]:
import nltk

Map the text data to the word_tokenize function in NLTK

In [6]:
words = data.flatMap(lambda x: nltk.word_tokenize(x))

Return ten of the words in the dataset

In [7]:
print words.take(10)

[u'Address', u'on', u'the', u'State', u'of', u'the', u'Union', u'Delivered', u'Before', u'a']


NTLK’s POS-tagger can be used to find the part of speech for each word

In [8]:
pos_word = words.map(lambda x: nltk.pos_tag([x]))

Return five of the words in the dataset

In [9]:
print pos_word.take(5)

[[(u'Address', 'NN')], [(u'on', 'IN')], [(u'the', 'DT')], [(u'State', 'NN')], [(u'of', 'IN')]]


In [32]:
places = data.map(lambda x: x[0] + "\t\t" + str(len(x[1])))

In [33]:
places.take(5)

[u'file:/usr/share/nltk_data/corpora/state_union/1961-Kennedy.txt\t\t35090',
 u'file:/usr/share/nltk_data/corpora/state_union/2001-GWBush-2.txt\t\t18036',
 u'file:/usr/share/nltk_data/corpora/state_union/1992-Bush.txt\t\t28674',
 u'file:/usr/share/nltk_data/corpora/state_union/1958-Eisenhower.txt\t\t30324',
 u'file:/usr/share/nltk_data/corpora/state_union/1994-Clinton.txt\t\t42133']